<a href="https://colab.research.google.com/github/Israt1063/3.Brain-Tumor-Detection-Using-Custom-CNN/blob/main/3_Brain_Tumor_Detection_Using_Custom_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Summary

Step 1: Import
Step 2: Dataset class
Step 3: Transforms
Step 4: DataLoaders
Step 5: CNN model
Step 6: Training loop
Step 7: Run training



In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Copy kaggle.json to right place
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Mounted at /content/drive


In [2]:
!kaggle datasets download -d navoneel/brain-mri-images-for-brain-tumor-detection


Dataset URL: https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection
License(s): copyright-authors
  0% 0.00/15.1M [00:00<?, ?B/s]
100% 15.1M/15.1M [00:00<00:00, 823MB/s]


In [3]:
!unzip brain-mri-images-for-brain-tumor-detection.zip -d brain_tumor_data


Archive:  brain-mri-images-for-brain-tumor-detection.zip
  inflating: brain_tumor_data/brain_tumor_dataset/no/1 no.jpeg  
  inflating: brain_tumor_data/brain_tumor_dataset/no/10 no.jpg  
  inflating: brain_tumor_data/brain_tumor_dataset/no/11 no.jpg  
  inflating: brain_tumor_data/brain_tumor_dataset/no/12 no.jpg  
  inflating: brain_tumor_data/brain_tumor_dataset/no/13 no.jpg  
  inflating: brain_tumor_data/brain_tumor_dataset/no/14 no.jpg  
  inflating: brain_tumor_data/brain_tumor_dataset/no/15 no.jpg  
  inflating: brain_tumor_data/brain_tumor_dataset/no/17 no.jpg  
  inflating: brain_tumor_data/brain_tumor_dataset/no/18 no.jpg  
  inflating: brain_tumor_data/brain_tumor_dataset/no/19 no.jpg  
  inflating: brain_tumor_data/brain_tumor_dataset/no/2 no.jpeg  
  inflating: brain_tumor_data/brain_tumor_dataset/no/20 no.jpg  
  inflating: brain_tumor_data/brain_tumor_dataset/no/21 no.jpg  
  inflating: brain_tumor_data/brain_tumor_dataset/no/22 no.jpg  
  inflating: brain_tumor_data/bra

In [4]:
import os
print(os.listdir("brain_tumor_data"))


['yes', 'no', 'brain_tumor_dataset']


In [29]:
#Step 1: Import necessary libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image


In [30]:
class BrainTumorDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))  # ['no', 'yes']
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}

        self.samples = []
        for cls in self.classes:
            cls_folder = os.path.join(root_dir, cls)
            for fname in os.listdir(cls_folder):
                path = os.path.join(cls_folder, fname)
                self.samples.append((path, self.class_to_idx[cls]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label


In [31]:
train_transforms = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet means, can change if needed
                         std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [32]:
dataset_root = "brain_tumor_data/brain_tumor_dataset"

train_dataset = BrainTumorDataset(dataset_root, transform=train_transforms)
val_dataset = BrainTumorDataset(dataset_root, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)


In [33]:
class BrainTumorCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(BrainTumorCNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # Input channels 3 for RGB
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 128x128 -> 64x64

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 64x64 -> 32x32

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 32x32 -> 16x16
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [34]:
def train_classifier(model, train_loader, val_loader, epochs, device):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        avg_val_loss = val_loss / len(val_loader)
        val_acc = 100 * correct / total

        print(f"Epoch {epoch+1}/{epochs} - "
              f"Train Loss: {avg_train_loss:.4f} - "
              f"Val Loss: {avg_val_loss:.4f} - "
              f"Val Accuracy: {val_acc:.2f}%")


In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BrainTumorCNN(num_classes=2)

train_classifier(model, train_loader, val_loader, epochs=10, device=device)


Epoch 1/10 - Train Loss: 0.9845 - Val Loss: 0.6934 - Val Accuracy: 45.85%
Epoch 2/10 - Train Loss: 0.6749 - Val Loss: 0.6147 - Val Accuracy: 72.33%
Epoch 3/10 - Train Loss: 0.6170 - Val Loss: 0.5620 - Val Accuracy: 71.54%
Epoch 4/10 - Train Loss: 0.5631 - Val Loss: 0.5305 - Val Accuracy: 73.12%
Epoch 5/10 - Train Loss: 0.5273 - Val Loss: 0.4719 - Val Accuracy: 80.63%
Epoch 6/10 - Train Loss: 0.4921 - Val Loss: 0.4384 - Val Accuracy: 81.03%
Epoch 7/10 - Train Loss: 0.4865 - Val Loss: 0.4640 - Val Accuracy: 82.21%
Epoch 8/10 - Train Loss: 0.4402 - Val Loss: 0.4017 - Val Accuracy: 81.42%
Epoch 9/10 - Train Loss: 0.4359 - Val Loss: 0.3901 - Val Accuracy: 82.21%
Epoch 10/10 - Train Loss: 0.4101 - Val Loss: 0.3523 - Val Accuracy: 84.58%


In [38]:
epoch_accuracies = [45.85, 72.33, 71.54, 73.12, 80.63, 81.03, 82.21, 81.42, 82.21, 84.58]
average_accuracy = sum(epoch_accuracies) / len(epoch_accuracies)
print(f"Average validation accuracy over 10 epochs: {average_accuracy:.2f}%")


Average validation accuracy over 10 epochs: 75.49%


In [36]:
torch.save(model.state_dict(), "brain_tumor_cnn.pth")


In [37]:
model = BrainTumorCNN(num_classes=2)
model.load_state_dict(torch.load("brain_tumor_cnn.pth"))
model.eval()


BrainTumorCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=32768, out_features=256, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=256, out_features=2, bias=True)
  )
)